In [1]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import ecc
import helper
import tx
import script

### Check a txn for positive fee

In [62]:
from tx import Tx
from io import BytesIO
raw_tx = ('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006b483045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed01210349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278afeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600')
stream = BytesIO(bytes.fromhex(raw_tx))
transaction = Tx.parse(stream)
print(transaction.fee() >= 0)

True


In [63]:
transaction

tx: 452c629d67e41baec3ac6f04fe744b4b9617f8f859c63b3002f8684e7a4fee03
version: 1
tx_ins:
d1c789a9c60383bf715f3f6ad9d14b91fe55f3deb369fe5d9280cb1a01793f81:0
tx_outs:
32454049:OP_DUP OP_HASH160 bc3b654dca7e56b04dca18f2566cdaf02e8d9ada OP_EQUALVERIFY OP_CHECKSIG
10011545:OP_DUP OP_HASH160 1c4bc762dd5423e332166702cb75f40df79fea12 OP_EQUALVERIFY OP_CHECKSIG
locktime: 410393

### Check a signature hash works (it's given in this case)

In [6]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
print(point.verify(z, signature))

True


In [8]:
from helper import hash256
modified_tx = bytes.fromhex('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000001976a914a802fc56c704ce87c42d7c92eb75e7896bdc41ae88acfeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac1943060001000000')
h256 = hash256(modified_tx)
z = int.from_bytes(h256, 'big')
print(hex(z))

0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6


In [9]:
stream2 = BytesIO(modified_tx)
transaction2 = Tx.parse(stream2)
transaction2

tx: 2e783ec25c8dfb008d9662f57a8a6ab26eda9fdcb778bb9111cf162a4c03550c
version: 1
tx_ins:
d1c789a9c60383bf715f3f6ad9d14b91fe55f3deb369fe5d9280cb1a01793f81:0
tx_outs:
32454049:OP_DUP OP_HASH160 bc3b654dca7e56b04dca18f2566cdaf02e8d9ada OP_EQUALVERIFY OP_CHECKSIG
10011545:OP_DUP OP_HASH160 1c4bc762dd5423e332166702cb75f40df79fea12 OP_EQUALVERIFY OP_CHECKSIG
locktime: 410393

In [10]:
from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
point.verify(z, signature)

True

### Exercise 1

Write the `sig_hash` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sig_hash`

In [11]:
# Exercise 1

reload(tx)
run(tx.TxTest("test_sig_hash"))

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


### Exercise 2

Write the `verify_input` method for the `Tx` class. You will want to use the `TxIn.script_pubkey()`, `Tx.sig_hash()` and `Script.evaluate()` methods.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_verify_p2pkh`

In [12]:
# Exercise 2

reload(tx)
run(tx.TxTest("test_verify_p2pkh"))

.
----------------------------------------------------------------------
Ran 1 test in 0.242s

OK


### Construct a TXN

#### pay 0.1tBTC from
#### mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2 
#### to
#### mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf

In [39]:
tx_in

0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299:13

In [36]:
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx
prev_tx = bytes.fromhex('0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299')
prev_index = 13
tx_in = TxIn(prev_tx, prev_index)
tx_outs = []
change_amount = int(0.33*100000000)
change_h160 = decode_base58('mzx5YhAH9kNHtcN481u6WkjeHjYtVeKVh2')
change_script = p2pkh_script(change_h160)
change_output = TxOut(amount=change_amount, script_pubkey=change_script)
target_amount = int(0.1*100000000)
target_h160 = decode_base58('mnrVtF8DWjMu839VW3rBfgYaAfKk8983Xf')
target_script = p2pkh_script(target_h160)
target_output = TxOut(amount=target_amount, script_pubkey=target_script)
tx_obj = Tx(1, [tx_in], [change_output, target_output], 0, True)
print(tx_obj)

tx: cd30a8da777d28ef0e61efe68a9f7c559c1d3e5bcd7b265c850ccb4068598d11
version: 1
tx_ins:
0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299:13
tx_outs:
33000000:OP_DUP OP_HASH160 d52ad7ca9b3d096a38e752c2018e6fbc40cdf26f OP_EQUALVERIFY OP_CHECKSIG
10000000:OP_DUP OP_HASH160 507b27411ccf7f16f10297de6cef3f291623eddf OP_EQUALVERIFY OP_CHECKSIG
locktime: 0


In [37]:
from ecc import PrivateKey
from helper import SIGHASH_ALL
z = transaction.sig_hash(0)
private_key = PrivateKey(secret=8675309)
der = private_key.sign(z).der()
sig = der + SIGHASH_ALL.to_bytes(1, 'big')
sec = private_key.point.sec()
script_sig = Script([sig, sec])
transaction.tx_ins[0].script_sig = script_sig 
print(transaction.serialize().hex())

0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf8303c6a989c7d1000000006a47304402207db2402a3311a3b845b038885e3dd889c08126a8570f26a844e3e4049c482a11022010178cdca4129eacbeab7c44648bf5ac1f9cac217cd609d216ec2ebc8d242c0a012103935581e52c354cd2f484fe8ed83af7a3097005b2f9c60bff71d35bd795f54b67feffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600


In [40]:
from ecc import PrivateKey
from helper import hash256, little_endian_to_int
secret = little_endian_to_int(hash256(b'Jimmy Song secret'))
private_key = PrivateKey(secret)
print(private_key.point.address(testnet=True))

mn81594PzKZa9K3Jyy1ushpuEzrnTnxhVg


In [42]:
tx_in

0d6fe5213c0b3291f208cba8bfb59b7476dffacc4e5cb66f6eb20a080843a299:13

### Exercise 3

Write the `sign_input` method for the `Tx` class.

#### Make [this test](/edit/code-ch07/tx.py) pass: `tx.py:TxTest:test_sign_input`

In [41]:
# Exercise 3

reload(tx)
run(tx.TxTest("test_sign_input"))

.
----------------------------------------------------------------------
Ran 1 test in 0.196s

OK


### Exercise 4

Create a testnet transaction that sends 60% of a single UTXO to `mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv`. The remaining amount minus fees should go back to your own change address. This should be a 1 input, 2 output transaction.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

In [59]:
# Exercise 4

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# create 1 TxIn and 2 TxOuts
# 1 of the TxOuts should be back to your address (mhePDRF4ZQHpug4bJm8dikKKv5MVVHFpg4)
# the other TxOut should be to this address
target_satoshis = 10_000

# get the private key from the exercise in Chapter 4
# change address should be the address generated from Chapter 4 (mhePDRF4ZQHpug4bJm8dikKKv5MVVHFpg4)
change_satoshis = 1822695 - target_satoshis - 1000

# get the prev_tx and prev_index from the transaction where you got
# some testnet coins (42fc4b277e09abffe535bb48e04b942c19a5fa6688412da384a43d8828f3841b : 1)
# create a transaction input for the previous transaction with
# the default ScriptSig and sequence
prev_tx = bytes.fromhex('240ea9f9d72c14b98b05d00b23c25cbb47348aaadfe5e7e42b726691d5f282b8')
prev_index = 0
tx_ins = []
tx_ins.append(TxIn(prev_tx, prev_index))

# target amount should be 60% of the output amount
# set the fee to some reasonable amount
# change amount = amount from the prev tx - target amount - fee
target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'
tx_outs = []
h160 = decode_base58(target_address)
script_pubkey = p2pkh_script(h160)
tx_outs.append(TxOut(target_satoshis, script_pubkey))

change_address = 'mhePDRF4ZQHpug4bJm8dikKKv5MVVHFpg4'
h160 = decode_base58(change_address)
script_pubkey = p2pkh_script(h160)
tx_outs.append(TxOut(change_satoshis, script_pubkey))

tx_obj = Tx(1, tx_ins, tx_outs, 0, testnet=True)

# create a transaction output for the target amount and address
# create a transaction output for the change amount and address
# create the transaction object

# sign the one input in the transaction object using the private key
# print the transaction's serialization in hex
# broadcast at http://testnet.blockchain.info/pushtx
passphrase = b'hooli HOOLI Chicken! Kapow!! 4893'
secret = little_endian_to_int(hash256(passphrase))
priv=PrivateKey(secret)
print(tx_obj.sign_input(0, priv))

True


In [60]:
tx_obj.serialize().hex()
# f3666ce32fc49b9c3337c28e02ed5376804b5e3a3f701090f6ad30fc72a390ed is the txid

'0100000001b882f2d59166722be4e7e5dfaa8a3447bb5cc2230bd0058bb9142cd7f9a90e24000000006b483045022100a27f81ad9bba804247b8f4b1f4372b03c1d43fd29ac9c2f21551b8270ae179c702206977f50124a36a4548026d6ff5bda21f2153021133a671d9c87fd009ab29c272012102af85c906ccdad9457e3b1860077696f77992f85521c4db806167a1cee0ca8126ffffffff0210270000000000001976a914ad346f8eb57dee9a37981716e498120ae80e44f788acefa41b00000000001976a9141757f72d70de6322f7695624b204ad3d5e799e6f88ac00000000'

### Exercise 5

Advanced: get some more testnet coins from a testnet faucet and create a 2 input, 1 output transaction. 1 input should be from the faucet, the other should be from the previous exercise, the output can be your own address.

You can broadcast the transaction at https://testnet.blockchain.info/pushtx

##### prev_txn1 f3666ce32fc49b9c3337c28e02ed5376804b5e3a3f701090f6ad30fc72a390ed : 1 (the one created above)
##### prev_txn2 d99656697bd43208ef1bb6e512842f1e6482ba25f887b4eb95d818d588dd18fa : 1 (from faucet)

In [66]:
# Exercise 5

from ecc import PrivateKey
from helper import decode_base58, SIGHASH_ALL
from script import p2pkh_script, Script
from tx import TxIn, TxOut, Tx

# Create 2 TxIns, 1 from the Exercise 4 and 1 from a testnet faucet
# Creat 1 TxOut to the address above
target_address = 'mhePDRF4ZQHpug4bJm8dikKKv5MVVHFpg4'
target_satoshis = 1811695 + 1863623 - 1000

# get the private key from the exercise in Chapter 4
passphrase = b'hooli HOOLI Chicken! Kapow!! 4893'
secret = little_endian_to_int(hash256(passphrase))
priv=PrivateKey(secret)

# get the prev_tx and prev_index from the transaction where you got
# some testnet coins
# create the first transaction input with the default ScriptSig and
# sequence
# get the prev_tx and prev_index from the transaction in Exercise 4
# create the second transaction input with the default ScriptSig and
# sequence
tx_ins = []

prev_tx1 = bytes.fromhex('f3666ce32fc49b9c3337c28e02ed5376804b5e3a3f701090f6ad30fc72a390ed')
prev_index1 = 1
tx_ins.append(TxIn(prev_tx1, prev_index1))

prev_tx2 = bytes.fromhex('d99656697bd43208ef1bb6e512842f1e6482ba25f887b4eb95d818d588dd18fa')
prev_index2 = 1
tx_ins.append(TxIn(prev_tx2, prev_index2))

# set the fee to some reasonable amount
# target amount should be the sum of the inputs - fee
# create a transaction output for the amount and address
tx_outs = []
h160 = decode_base58(target_address)
script_pubkey = p2pkh_script(h160)
tx_outs.append(TxOut(target_satoshis, script_pubkey))


# sign the first input using the private key

# sign the second input using the private key 

# print the transaction's serialization in hex
tx_obj = Tx(1, tx_ins, tx_outs, 0, testnet=True)

tx_obj.sign_input(0, priv)
tx_obj.sign_input(1, priv)
print(tx_obj)
# broadcast at http://testnet.blockchain.info/pushtx
tx_obj.serialize().hex()

tx: f1ae466d0892e1b4c8d254f36bdf4b29578f561263be81fd97db3a5d941ea104
version: 1
tx_ins:
f3666ce32fc49b9c3337c28e02ed5376804b5e3a3f701090f6ad30fc72a390ed:1
d99656697bd43208ef1bb6e512842f1e6482ba25f887b4eb95d818d588dd18fa:1
tx_outs:
3674318:OP_DUP OP_HASH160 1757f72d70de6322f7695624b204ad3d5e799e6f OP_EQUALVERIFY OP_CHECKSIG
locktime: 0


'0100000002ed90a372fc30adf69010703f3a5e4b807653ed028ec237339c9bc42fe36c66f3010000006a47304402205257dd388df6474297f7c8ed0ac08bcb5bc7c85efed10f4cd1257da173498eb4022017bed0095b8871fa10dca8cfc7cc488985e63b4befd108df2a0e0c3009f2ce51012102af85c906ccdad9457e3b1860077696f77992f85521c4db806167a1cee0ca8126fffffffffa18dd88d518d895ebb487f825ba82641e2f8412e5b61bef0832d47b695696d9010000006a47304402205b677cd42ab1d04790f19401790bf16f70b6e876416b5c674f68c3cd4fd83f8402203b5d4587d84025c6286d32f35d203cc5b7c849e1de131cffe7a839c8c6d31de1012102af85c906ccdad9457e3b1860077696f77992f85521c4db806167a1cee0ca8126ffffffff01ce103800000000001976a9141757f72d70de6322f7695624b204ad3d5e799e6f88ac00000000'

#### txnid - f1ae466d0892e1b4c8d254f36bdf4b29578f561263be81fd97db3a5d941ea104